In [1]:
import numpy as np
import tensorflow as tf
import import_ipynb
import glove_batch_generator

importing Jupyter notebook from glove_batch_generator.ipynb


In [2]:
batch_size=32
learning_rate=0.5
window_size = 3
vector_len = 50
vocab_size = 33702
steps = vocab_size**2
max_x=100.0
alpha=0.75
val_indexes = [445,633,4904,16842,1478]
#device_name = '/gpu:0'

In [3]:
def glove():
    Xs = tf.placeholder(tf.int32 , [batch_size,])
    Ys = tf.placeholder(tf.int32 , [batch_size,])
    cooccur = tf.placeholder (tf.float32 , [batch_size,])
    
    with tf.variable_scope ('glove') as scope:
        Vs = tf.Variable(tf.random_uniform([vocab_size,vector_len] , -1.0 , 1.0))
        Us = tf.Variable(tf.random_uniform([vocab_size,vector_len] , -1.0 , 1.0))
        
        V_bias = tf.Variable(tf.random_uniform([vocab_size] , -1.0 , 1.0))
        U_bias = tf.Variable(tf.random_uniform([vocab_size] , -1.0 , 1.0))
        
        X_embd = tf.nn.embedding_lookup(Vs , Xs)
        Y_embd = tf.nn.embedding_lookup(Us , Ys)
        
        X_bias = tf.nn.embedding_lookup(V_bias , Xs)
        Y_bias = tf.nn.embedding_lookup(U_bias , Ys)
        
        fs = tf.minimum (1.0 , tf.pow(tf.div(tf.to_float(cooccur) , max_x) ,alpha))
        
        log_cooccur = tf.log((cooccur))
        
        prod = tf.reduce_sum (tf.multiply(X_embd , Y_embd), axis=1)
        
        
        dist = tf.square(tf.add_n([prod , X_bias , Y_bias , (log_cooccur)]))
        
        loss = tf.multiply(fs , dist)
        
        total_loss = tf.reduce_mean(loss)
        
        new_embd = tf.add(Us , Vs)/2
        
        val_embd = tf.identity(new_embd)
        similarity = tf.matmul(val_embd , new_embd , transpose_b=True)
        
        return Xs , Ys , cooccur  ,  total_loss , new_embd , similarity 

In [4]:
def generator():
    yield from glove_batch_generator.gen_batch(window_size , batch_size , vocab_size)

In [5]:
def run():
    with open("indexWords.txt" , "rt") as infile:
        index_words = eval(infile.read())
        
        
    data = tf.data.Dataset.from_generator(generator , (tf.int32 , tf.int32 , tf.int32))
    iterator = data.make_initializable_iterator()
    first , second , cooc = iterator.get_next()
    
  
   # with tf.device(device_name):
    Xs , Ys , cooccur , total_loss  , new_embd , similarity = glove()
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
    
    with tf.Session() as sess:
        sess.run(iterator.initializer)
        sess.run(tf.global_variables_initializer())
        for i in range (steps):
            try:
                batch_loss , _ = sess.run([ total_loss , optimizer] , feed_dict={Xs: first.eval() , Ys: second.eval() , cooccur: cooc.eval() })
                    
                if i%1000 ==0:
                    print(str(i)+":"+str(batch_loss))
                

                if i%5000 == 0:
                    sim = similarity.eval()

                    for j in val_indexes:
                        listt = (-sim[j,:]).argsort()[1:6]
                        print(listt)
                        print (index_words[j])
                        list2 = [index_words[k] for k in listt]
                        print (list2)



            except tf.errors.OutOfRangeError:
                sess.run(iterator.initializer)
            

In [ ]:
if __name__ == '__main__':
    run()

0:0.3832074
[13708 10279   633 20304 15471]
two
['natufian', 'unfair', 'person', 'regiment', 'uc']
[10279 30121   884 27313 17074]
person
['unfair', 'sadler', 'fail', 'deschauenseei', 'footed']
[ 6197  7558 11149 21824 10108]
england
['realm', 'baruch', 'crunches', 'vectors', 'burials']
[31497 33219   538 29730 17107]
week
['procure', 'reassuring', 'bakunin', 'stonewall', 'conveyor']
[ 2911 27949 31718 32150 28170]
big
['hypothesis', 'hanseatic', 'galaxy', 'unicorn', 'intervehicle']
1000:0.2602082
2000:0.16086784
3000:0.20905596
